In [8]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

def extract_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(rgb_frame)

        if result.pose_landmarks:
            landmarks = [(lm.x, lm.y, lm.z) for lm in result.pose_landmarks.landmark]
            keypoints_list.append(landmarks)

    cap.release()
    return keypoints_list

def normalize_keypoints(keypoints):
    # Normalize to a common reference point, e.g., shoulder-hip line
    normalized_keypoints = []
    for frame in keypoints:
        shoulder = frame[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        hip = frame[mp_pose.PoseLandmark.LEFT_HIP.value]

        ref_vector = np.array(shoulder) - np.array(hip)
        scale = np.linalg.norm(ref_point)
        normalized_frame = [(kp[0] - shoulder[0], kp[1] - shoulder[1]) for kp in frame]
        normalized_keypoints.append(normalized_keypoints)

    return normalized_keypoints

def calculate_joint_angles(keypoints):
    # Example: elbow and shoulder angles
    angles = []
    for frame in keypoints:
        shoulder = np.array(frame[mp_pose.PoseLandmark.LEFT_SHOULDER])
        elbow = np.array(frame[mp_pose.PoseLandmark.LEFT_ELBOW])
        wrist = np.array(frame[mp_pose.PoseLandmark.LEFT_WRIST])

        # Calculate angle at elbow
        upper_arm = shoulder - elbow
        wrist = np.array(frame[mp_pose.PoseLandmark.LEFT_WRIST])
        elbow_angle = angle_between_joints(shoulder, elbow, wrist)
        angles.append(elbow_angle)

    return angles

def angle_between_joints(a, b, c):
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(ba @ bc / (np.linalg.norm(ba) * np.linalg.norm(bc)))
    return np.degrees(angle)

def compare_forehands(amateur_angles, pro_angles):
    # Compute differences frame-by-frame
    differences = [abs(a - p) for a, p in zip(amateur_angles, pro_angles)]
    avg_difference = np.mean(differences)
    return avg_difference, differences

def compare_joint_angles(amateur_angles, pro_angles):
    # Ensure both angle lists are the same length by taking the shorter one
    min_length = min(len(amateur_angles), len(pro_angles))
    amateur_angles = amateur_angles[:min_length]
    pro_angles = pro_angles[:min_length]

    # Calculate differences between amateur and pro angles frame-by-frame
    differences = [abs(a - p) for a, p in zip(amateur_angles, pro_angles)]
    avg_difference = np.mean(differences)

    return avg_difference, differences


# Plotting joint angle differences
def plot_angle_differences(differences):
    frames = np.arange(1, len(differences) + 1)

    plt.figure(figsize=(12, 6))
    plt.plot(frames, differences, marker='o')
    plt.title('Joint Angle Differences (Amateur vs. Pro Forehand)')
    plt.xlabel('Frame Number')
    plt.ylabel('Angle Difference (degrees)')
    plt.axhline(y=np.mean(differences), color='red', linestyle='--', label=f'Average Difference: {np.mean(differences):.2f}°')

    # Highlight key frames with highest differences
    key_frames_idx = np.argsort(differences)[-3:]  # top 3 key frames
    for idx in key_frames_idx:
        plt.annotate(f'Frame {idx+1}\n({differences[idx]:.1f}°)',
                     (frames[idx], differences[idx]),
                     textcoords="offset points",
                     xytext=(0,10),
                     ha='center',
                     fontsize=9,
                     arrowprops=dict(facecolor='red', shrink=0.05))

    plt.title('Joint Angle Differences (Amateur vs Pro)')
    plt.xlabel('Frame Number')
    plt.ylabel('Angle Difference (degrees)')
    plt.legend()
    plt.grid(True)
    plt.show()


I0000 00:00:1741656162.081714 55239860 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro


W0000 00:00:1741656162.160793 55250628 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741656162.171751 55250628 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [11]:
# Call this function after your previous calculation:
plot_angle_differences(diff_list)

# Example usage:
amateur_keypoints = extract_keypoints('mohammad-alhomaid-forehands.mov')
pro_keypoints = extract_keypoints('jannik-sinner-forehands.mp4')

amateur_angles = calculate_joint_angles(amateur_keypoints)
pro_angles = calculate_joint_angles(pro_keypoints)


avg_diff, diff_list = compare_joint_angles(amateur_angles, pro_angles)
print(f"Average technique difference: {avg_diff:.2f} degrees")


NameError: name 'diff_list' is not defined